In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
from hmmlearn import hmm
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = '../../data/HI-Small_Trans.csv'
df = pd.read_csv(data_path)
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [3]:
# necessary preprocessing
df['FromAccount'] = df['From Bank'].astype(str) + '_' + df['Account']
df['ToAccount'] = df['To Bank'].astype(str) + '_' + df['Account.1']
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,FromAccount,ToAccount
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0,10_8000EBD30,10_8000EBD30
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,3208_8000F4580,1_8000F5340
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0,3209_8000F4670,3209_8000F4670
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0,12_8000F5030,12_8000F5030
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0,10_8000F5200,10_8000F5200


In [4]:
# Encoding
encode_curr = LabelEncoder().fit(pd.concat([df['Receiving Currency'], df['Payment Currency']], ignore_index=True)) # For all Currency 
encode_paym_format = LabelEncoder().fit(df['Payment Format']) # Payment Format
encode_acct = LabelEncoder().fit(pd.concat([df['FromAccount'], df['ToAccount']], ignore_index=True)) # For all unique Account
encode_bank = LabelEncoder().fit(pd.concat([df['From Bank'], df['To Bank']], ignore_index=True)) # For all unique Bank codes


In [5]:
%%time

clean_df = pd.DataFrame()

clean_df['Timestamp'] = pd.to_datetime(df['Timestamp'])

clean_df['FromAccount'] = encode_acct.transform(df['FromAccount'])
clean_df['ToAccount'] = encode_acct.transform(df['ToAccount'])
clean_df['FromBank'] = encode_bank.transform(df['From Bank'])
clean_df['ToBank'] = encode_bank.transform(df['To Bank'])

clean_df['ReceivingCurrency'] = encode_curr.transform(df['Receiving Currency'])
clean_df['PaymentCurrency'] = encode_curr.transform(df['Payment Currency'])
clean_df['PaymentFormat'] = encode_paym_format.transform(df['Payment Format'])
clean_df['AmountPaid'] = df['Amount Paid']
clean_df['AmountReceived'] = df['Amount Received']
clean_df['FromBankOriginal'] = df['From Bank']
clean_df['ToBankOriginal'] = df['To Bank']
clean_df['FromAccountOriginal'] = df['Account']
clean_df['ToAccountOriginal'] = df['Account.1']
clean_df['ReceivingCurrencyOriginal'] = df['Receiving Currency']
clean_df['PaymentCurrencyOriginal'] = df['Payment Currency']
clean_df['PaymentFormatOriginal'] = df['Payment Format']
clean_df['IsLaundering'] = df['Is Laundering']

clean_df.head()


CPU times: total: 8.11 s
Wall time: 8.11 s


,Timestamp,FromAccount,ToAccount,FromBank,ToBank,ReceivingCurrency,PaymentCurrency,PaymentFormat,AmountPaid,AmountReceived,FromBankOriginal,ToBankOriginal,FromAccountOriginal,ToAccountOriginal,ReceivingCurrencyOriginal,PaymentCurrencyOriginal,PaymentFormatOriginal,IsLaundering
0,2022-09-01 00:20:00,6530,6530,8,8,12,12,5,3697.34,3697.34,10,10,8000EBD30,8000EBD30,US Dollar,US Dollar,Reinvestment,0
1,2022-09-01 00:20:00,358174,176809,109,0,12,12,3,0.01,0.01,3208,1,8000F4580,8000F5340,US Dollar,US Dollar,Cheque,0
2,2022-09-01 00:00:00,358476,358476,110,110,12,12,5,14675.57,14675.57,3209,3209,8000F4670,8000F4670,US Dollar,US Dollar,Reinvestment,0
3,2022-09-01 00:02:00,74640,74640,10,10,12,12,5,2806.97,2806.97,12,12,8000F5030,8000F5030,US Dollar,US Dollar,Reinvestment,0
4,2022-09-01 00:06:00,6538,6538,8,8,12,12,5,36682.97,36682.97,10,10,8000F5200,8000F5200,US Dollar,US Dollar,Reinvestment,0


In [7]:
clean_df.describe()

,Timestamp,FromAccount,ToAccount,FromBank,ToBank,ReceivingCurrency,PaymentCurrency,PaymentFormat,AmountPaid,AmountReceived,IsLaundering
count,5078345,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06
mean,2022-09-05 07:16:08.194274816,2.382680e+05,2.123980e+05,1.948130e+03,2.637933e+03,8.382732e+00,8.413146e+00,3.042442e+00,4.509273e+06,5.988726e+06,1.019427e-03
min,2022-09-01 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-06,1.000000e-06,0.000000e+00
25%,2022-09-02 04:32:00,1.044850e+05,9.603400e+04,3.300000e+01,5.400000e+02,4.000000e+00,4.000000e+00,3.000000e+00,1.844800e+02,1.833700e+02,0.000000e+00
50%,2022-09-05 12:16:00,2.046570e+05,1.928340e+05,5.960000e+02,8.360000e+02,1.000000e+01,1.000000e+01,3.000000e+00,1.414540e+03,1.411010e+03,0.000000e+00
75%,2022-09-08 03:13:00,3.625850e+05,2.899310e+05,9.710000e+02,6.111000e+03,1.200000e+01,1.200000e+01,4.000000e+00,1.229784e+04,1.234627e+04,0.000000e+00
max,2022-09-18 16:18:00,5.150870e+05,5.150870e+05,3.046900e+04,3.046400e+04,1.400000e+01,1.400000e+01,6.000000e+00,1.046302e+12,1.046302e+12,1.000000e+00
std,NaN,1.633302e+05,1.443685e+05,3.564369e+03,3.030044e+03,4.121243e+00,4.120945e+00,1.489543e+00,8.697728e+08,1.037183e+09,3.191219e-02


In [6]:
clean_df.to_csv(f'HI-Small_Trans_processed_w_original.csv', index=False)